# 隐马尔可夫模型 HMM

In [1]:
import numpy as np

In [2]:
class HiddenMarkov:
    def forward(self, Q, V, A, B, O, PI):
        """
        前向算法
        Q: 状态集合
        V: 观测集合
        A: 状态转移概率矩阵
        B: 观测概率矩阵
        O: 观测序列
        PI: 初始概率分布
        """
        
        # 存在的状态数量
        N = len(Q)  
        # 观测序列长度
        M = len(O)
        
        #
        alphas = np.zeros((N, M))
        
        # 有几个时刻，有几个观测序列，
        T = M
        
        # 遍历每一个时刻，计算alpha值
        for t in range(T):
            # 找出序列对应的索引
            indexOfO = V.index(O[t])
            for i in range(N):
                # 初始值 10.15
                if t == 0:
                    alphas[i][t] = PI[t][i] * B[i][indexOfO]
                    print(' alpha1(%d) = p%db%db(o1)=%f' % (i, i, i, alphas[i][t]))
                else:
                    # 10.16
                    alphas[i][t] = np.dot([alpha[t-1] for alpha in alphas], [a[i] for a in A]) * B[i][indexOfO]
                    print(' alpha%d(%d)=[sigma alpha%d(i)ai%d]b%d(o%d)=%f' % (t, i, t - 1, i, i, t, alphas[i][t]))
        
        P = np.sum([alpha[M - 1] for alpha in alphas])
        
        
    def backword(self, Q, V, A, B, O, PI):
        N = len(Q)
        M = len(O)
        betas = np.ones((N, M))
        
        for i in range(N):
            print('beta%d(%d)=1' % (M, i))
        
        for t in range(M - 2, -1, -1):
            indexOfO = V.index(O[t + 1])
            for i in range(N):
                betas[i][t] = np.dot(np.multiply(A[i], [b[indexOfO] for b in B]), [beta[t + 1] for beta in betas])
                realT = t + 1
                realI = i + 1
                print('beta%d(%d)=[sigma a%djbj(o%d)beta%d(j)] = (' % (realT, realI, realI, realT + 1, realT + 2), end = ' ')
                for j in range(N):
                    print("%.2f*%.2f*%.2f + " % (A[i][j], B[j][indexOfO], betas[j][t+1]), end=' ')
                print("0)=%.3f" % betas[i][t])
                
        
        indexOfO = V.index(O[0])
        P = np.dot(np.multiply(PI, [b[indexOfO] for b in B]), [beta[0] for beta in betas])
        print("P(O|lambda)=", end=" ")
        
        for i in range(N):
            print("%.1f*%.1f*%.5f+" % (PI[0][i], B[i][indexOfO], betas[i][0]))
        print("O=%f" % P)
        
    def viterbi(self, Q, V, A, B, O, PI):
        N = len(Q)
        M = len(O)
        deltas = np.zeros((N, M))
        psis = np.zeros((N, M))
        I = np.zeros((1, M))
        
        for t in range(M):
            realT = t +1
            indexOfO = V.index(O[t])
            for i in range(N):
                realI = i + 1
                if t == 0:
                    deltas[i][t] = PI[0][i] * B[i][indexOfO]
                    psis[i][t] = 0
                    print('delta1(%d)=pi%d * b%d(o1) = %.2f * %.2f = %.2f' % (realI, realI, realI, PI[0][i], B[i][indexOfO], deltas[i][t]))
                    print('psis1(%d)=0' % (realI))
                else:
                    deltas[i][t] = np.max(np.multiply([delta[t-1] for delta in deltas], [a[i] for a in A])) * B[i][indexOfO]
                    print("delta%d(%d)=max[delta%d(j)aj%d]b%d(o%d)=%.2f*%.2f=%.5f" % (realT, realI, realT-1, realI, realI, realT, 
                                np.max(np.multiply([delta[t-1] for delta in deltas], [a[i] for a in A])), B[i][indexOfO], deltas[i][t]))
                    psis[i][t] = np.argmax(np.multiply([delta[t-1] for delta in deltas], [a[i] for a in A])) + 1
                    print('psis%d(%d)=argmax[delta%d(j)aj%d]=%d' % (realT, realI, realT - 1, realI, psis[i][t]))
        
        print(deltas)
        print(psis)
        I[0][M-1] = np.argmax([delta[M - 1] for delta in deltas]) + 1
        print(" i %d = argmax[deltaT(i)]=%d" % (M, I[0][M-1]))
        
        for t in range(M - 2, -1, -1):
            I[0][t] = psis[int(I[0][t + 1]) - 1][t + 1]
            print("i%d=psis%d(i%d)=%d" % (t + 1, t + 2, t+2, I[0][t]))
        print("状态序列I: ", I)

In [3]:
# eg. 10.1
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['红', '白', '红', '白']
PI = [[0.2, 0.4, 0.4]]


In [4]:
HMM = HiddenMarkov()
HMM.backword(Q, V, A, B, O, PI)

beta4(0)=1
beta4(1)=1
beta4(2)=1
beta3(1)=[sigma a1jbj(o4)beta5(j)] = ( 0.50*0.50*1.00 +  0.20*0.60*1.00 +  0.30*0.30*1.00 +  0)=0.460
beta3(2)=[sigma a2jbj(o4)beta5(j)] = ( 0.30*0.50*1.00 +  0.50*0.60*1.00 +  0.20*0.30*1.00 +  0)=0.510
beta3(3)=[sigma a3jbj(o4)beta5(j)] = ( 0.20*0.50*1.00 +  0.30*0.60*1.00 +  0.50*0.30*1.00 +  0)=0.430
beta2(1)=[sigma a1jbj(o3)beta4(j)] = ( 0.50*0.50*0.46 +  0.20*0.40*0.51 +  0.30*0.70*0.43 +  0)=0.246
beta2(2)=[sigma a2jbj(o3)beta4(j)] = ( 0.30*0.50*0.46 +  0.50*0.40*0.51 +  0.20*0.70*0.43 +  0)=0.231
beta2(3)=[sigma a3jbj(o3)beta4(j)] = ( 0.20*0.50*0.46 +  0.30*0.40*0.51 +  0.50*0.70*0.43 +  0)=0.258
beta1(1)=[sigma a1jbj(o2)beta3(j)] = ( 0.50*0.50*0.25 +  0.20*0.60*0.23 +  0.30*0.30*0.26 +  0)=0.112
beta1(2)=[sigma a2jbj(o2)beta3(j)] = ( 0.30*0.50*0.25 +  0.50*0.60*0.23 +  0.20*0.30*0.26 +  0)=0.122
beta1(3)=[sigma a3jbj(o2)beta3(j)] = ( 0.20*0.50*0.25 +  0.30*0.60*0.23 +  0.50*0.30*0.26 +  0)=0.105
P(O|lambda)= 0.2*0.5*0.11246+
0.4*0.4*0.12174+
0.

In [5]:
HMM.forward(Q, V, A, B, O, PI)

 alpha1(0) = p0b0b(o1)=0.100000
 alpha1(1) = p1b1b(o1)=0.160000
 alpha1(2) = p2b2b(o1)=0.280000
 alpha1(0)=[sigma alpha0(i)ai0]b0(o1)=0.077000
 alpha1(1)=[sigma alpha0(i)ai1]b1(o1)=0.110400
 alpha1(2)=[sigma alpha0(i)ai2]b2(o1)=0.060600
 alpha2(0)=[sigma alpha1(i)ai0]b0(o2)=0.041870
 alpha2(1)=[sigma alpha1(i)ai1]b1(o2)=0.035512
 alpha2(2)=[sigma alpha1(i)ai2]b2(o2)=0.052836
 alpha3(0)=[sigma alpha2(i)ai0]b0(o3)=0.021078
 alpha3(1)=[sigma alpha2(i)ai1]b1(o3)=0.025188
 alpha3(2)=[sigma alpha2(i)ai2]b2(o3)=0.013824


In [6]:
HMM.viterbi(Q, V, A, B, O, PI)

delta1(1)=pi1 * b1(o1) = 0.20 * 0.50 = 0.10
psis1(1)=0
delta1(2)=pi2 * b2(o1) = 0.40 * 0.40 = 0.16
psis1(2)=0
delta1(3)=pi3 * b3(o1) = 0.40 * 0.70 = 0.28
psis1(3)=0
delta2(1)=max[delta1(j)aj1]b1(o2)=0.06*0.50=0.02800
psis2(1)=argmax[delta1(j)aj1]=3
delta2(2)=max[delta1(j)aj2]b2(o2)=0.08*0.60=0.05040
psis2(2)=argmax[delta1(j)aj2]=3
delta2(3)=max[delta1(j)aj3]b3(o2)=0.14*0.30=0.04200
psis2(3)=argmax[delta1(j)aj3]=3
delta3(1)=max[delta2(j)aj1]b1(o3)=0.02*0.50=0.00756
psis3(1)=argmax[delta2(j)aj1]=2
delta3(2)=max[delta2(j)aj2]b2(o3)=0.03*0.40=0.01008
psis3(2)=argmax[delta2(j)aj2]=2
delta3(3)=max[delta2(j)aj3]b3(o3)=0.02*0.70=0.01470
psis3(3)=argmax[delta2(j)aj3]=3
delta4(1)=max[delta3(j)aj1]b1(o4)=0.00*0.50=0.00189
psis4(1)=argmax[delta3(j)aj1]=1
delta4(2)=max[delta3(j)aj2]b2(o4)=0.01*0.60=0.00302
psis4(2)=argmax[delta3(j)aj2]=2
delta4(3)=max[delta3(j)aj3]b3(o4)=0.01*0.30=0.00220
psis4(3)=argmax[delta3(j)aj3]=3
[[0.1      0.028    0.00756  0.00189 ]
 [0.16     0.0504   0.01008  0.003024]
 